# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [3]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [4]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
#last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#while True:
#    '''
#    Your code here
#    
#    提示：可參考以下的Stack Overflow: 
#    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
#    '''

last_height = browser.execute_script("return document.body.scrollHeight")

while True:

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 英國保誠人壽攜手王道銀行　打造網路投保！「商品組合+購物車」一次滿足保險需求

▲英國保誠人壽。（圖／英國保誠人壽提供）
記者楊絡懸／台北報導
英國保誠人壽23日宣布，即日起與王道銀行合作，用戶可以透過王道銀行App或網路銀行購買保險商品，包括選購、試算、投保與核保等功能，都可在網路上直接處理，一次滿足所有需求；而保險商品如意外險，也可因應需求分別購買，另有定期壽險、醫療險及旅平險供選擇，提供全方位保障。
英國保誠人壽總經理王慰慈表示，王道銀行是台灣第一家原生數位銀行，在數位金融服務上，累積了豐富的客戶經驗，也積極地拓展年輕市場，與王道銀行攜手共同推出網路投保活動，正是彰顯英國保誠運用科技以為客戶創造更佳服務的承諾。
▲英國保誠人壽總經理王慰慈。（圖／記者湯興漢攝）
王慰慈說，英國保誠人壽致力讓客戶更輕鬆、便利獲得保險保障，「我們了解客戶的需要，特別是身處數位時代的千禧一族，尋求簡單、迅速、且貼合日益繁忙生活的解決方案，輕鬆從網路上了解商品，享受無縫點到點的購買體驗；面對客戶在數位時代不斷變化的需求，我們的首要任務是力求創新，以幫助台灣民眾保障健康及增長財富。」
王道銀行表示，王道銀行致力提供便利、安全、貼心的數位金融服務，希望藉由數位金融低門檻、無疆界、零時差的特色，為消費者生活創造更多便利性。王道銀行很高興與英國保誠人壽合作，提供消費者更完整的網路投保服務，使王道銀行的產品線更多樣與完善。
▲王道銀行。（圖／《ETtoday新聞雲》資料照）
英國保誠人壽表示，網路投保平台提供「碎片化商品組合」，民眾可自行選擇符合需求的投保商品並且自由組合，再透過獨創的「保險購物車」功能，將符合需求的商品隨手放進購物車，簡單選購投保；防疫期間，減少出門的風險，可以透過網路投保購買保障。
此外，英國保誠人壽與王道銀行也推出「王道有禮」網路投保活動，即日起至5月31日，王道銀行客戶透過王道銀行App或網路銀行的「O速保專區」，連結至英國保誠人壽網路投保，並以王道銀行簽帳金融卡進行繳費，除了享王道銀行簽帳金融卡1%的現金回饋，還可額外再享3%的現金回饋，刷卡繳保費的現金回饋共計4%，回饋金額無上限。
------------------

▲ 金管會顧立雄（右四）及證期局局長王詠心（左四），赴臺灣證券交易所視察撮合系統之運作情況，由證交所董事長許璋瑤（右三）及總經理簡立忠（右二）接待。（圖／證交所）
記者李瑞瑾／綜合報導
臺灣證券市場於今（23）日起實施盤中全面逐筆交易，證交所表示，實施首日證交所及證券商系統皆正常運作，經統計，總委託筆數計1,341萬6,649筆，其中限價委託占99.67%，市價委託占0.33%，顯示投資人仍以使用限價委託為主。
證交所指出，為關心新制實施首日交易系統運作情形，金管會主委顧立雄在證期局局長王詠心陪同下，赴證交所視察撮合系統之運作情況，由證交所董事長許璋瑤接待。證交所表示，實施首日交易系統均順利運行，配合逐筆交易之相關重大系統亦順利上線。
▲ 逐筆交易上路首日，證交所總經理簡立忠赴元大證券視察。（圖／證交所）
另外，證交所總經理簡立忠及櫃買中心總經理李愛玲率相關經理人親赴全台北中南等地共8家證券商營業處所（元大、富邦、凱基、群益金鼎、永豐金、大昌、光和及致和）瞭解新制上線市場運作情形，透過實地訪察，瞭解證券商從業人員接單及下單作業。
證交所指出，本次盤中逐筆交易新制順利施行，將持續關注整體市場是否有結構性變化，例如觀察大型股或中小型股的成交量及價格波動情形；分析整體市場投資人結構是否改變，並追蹤演算法交易或高頻交易的發展情形，必要時擬定相關規範予以控管，以維持市場結構的均衡，保障投資大眾權益。
----------------------------------------------------------------------

[12] 歐股下跌！　英、法、德股早盤重挫逾3％

▲歐洲股市本週開盤仍持續下墜。（圖／路透）
記者謝仁傑／綜合報導
新冠肺炎（COVID-19）疫情在歐美衝擊擴散，歐股上週五收盤也紛紛來到一個月以來的新低點。今（23）日早盤仍持續下墜，英、法、德股市目前皆重挫4％。
美股短短一個月自高點以來跌幅已3成，歐股也不遑多讓，英股一個月來下跌近3成，法股、德股皆下跌約3成5，西股下跌約3成3。
歐股今日表現（截至台灣時間下午6點07分）
英國FTSE100指數下跌229點，至4,961點，跌幅4.68％。
法國CAC40指數下跌129點，至3,919點，跌幅3.33％。
德國DAX指數下跌284點，至8,644點，跌幅3.33％。
西班牙IBE

▲法德藥經營團隊，左起為創辦人黃逸斌、董事長詹惠如、執行長鍾裕民。（圖／法德藥提供）
記者姚惠茹／台北報導
特殊學名藥大廠法德藥（4191）今（23）日代理的利巴韋林吸入劑（Ribavirin for Inhalation Solution, USP, ANDA207366），可治療下呼吸道病毒感染，是美國FDA唯一核准的學名藥吸入劑型，效用更優於口服與注射劑型，被列為新冠肺炎推薦用藥。
法德藥表示，目前正向中國國家藥品監督管理局（NMPA）申請緊急註冊，可望獲得加速審批，並且已經在中國大陸找到代理商，為利巴韋林吸入劑上市銷售做好準備，期望能盡速供應新冠肺炎的治療需求。
法德藥指出，新型冠狀病毒目前沒有可治療的特異性藥物，因此「老藥新用」成為短期內一種經濟和有效的療法開發策略，而與瑞德西韋（remdesivir）同屬廣譜核苷類抗病毒藥物的利巴韋林，能抑制肌苷酸-5-磷酸脫氫酶，阻斷肌苷酸轉化為鳥苷酸，從而抑制病毒的RNA和DNA合成，對DNA病毒和RNA病毒均有抑制複製作用。
中國國家衛健委所公告「新型冠狀病毒肺炎診療方案（試行第六版）」中提到，新冠病毒患者在臨床診療上可採取利巴韋林作為抗病毒治療策略（建議與干擾素或洛匹那韋/利托那韋合併使用），從學理上也支援利巴韋林於新冠肺炎的治療應用。
同時在中國呼吸與危重監護雜誌 2020年3月第19卷第2期 <針對新型冠狀病毒感染患者的霧化吸入治療的建議> 中，推薦具霧化專用劑型的利巴韋林霧化吸入治療法，而且利巴韋林也被載明於美國化學會（American Chemistry Society, ACS）推薦的候選療法報告中。
法德藥表示，公司擁有利巴韋林吸入劑的台灣與大陸地區總代理，其所代理的利巴韋林凍乾粉吸入劑為專供霧化吸入使用，目前利巴韋林在中國大陸僅有針劑，並無吸入劑，法德藥已向中國NMPA申請緊急註冊，可望獲得加速審批，提供新冠肺炎病患抗病毒藥物的另一選擇。
法德藥強調，致力於研發與生產高門檻的緩控釋製劑及利基型藥品，並發揮中美雙報的優勢，其中抗思覺失調症用藥富馬酸喹硫平緩釋片（Quetiapine fumarate ER tablet）已取得中、美兩國藥證，尤其去年底於中國開始出貨之後，目前已取得超過十個省級行政區的掛網公示，獲得二十家以上的醫院採購訂單，每個月出貨倍增，營收明顯增長。
此外，法德藥表示，重磅藥物長效

▲ 新台幣兌美元收盤貶值1.03角。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（23）天收盤貶值1.03角、收在30.405元兌1美元，成交金額14.15億美元。
新台幣匯率今天開盤貶值1.48角、以30.45元兌1美元開出，早盤一度重貶1.58角，來到最低價位30.46元，隨後位在30.4元附近震盪；今天最高價則是30.347元兌1美元。
新台幣匯率終場貶值1.03角、收在30.405元兌1美元。
根據央行發布之主要貨幣對美元匯率變動資料顯示，新台幣僅貶值0.34%，其它貨幣對美元匯率，澳幣貶值2.9%、韓元貶值1.58%、泰銖貶值1.43%、英鎊貶值1.4%、歐元貶值1.12%、日圓貶值0.64%、人民幣貶值0.63%。
另一方面，台股今日也受到重傷，今天收盤下跌344點，收在8890點，失守9000點關卡。
----------------------------------------------------------------------

[30] 富達調查：大陸可望率先反彈復甦　科技產業最抗疫

▲ 市場分析師指出，大陸經濟可望比其他市場率先復甦。（圖／達志影像／美聯社）
記者李瑞瑾／綜合報導
富達國際最新一項市場脈搏調查訪問了來自全球不同地區152位富達國際研究股票和債券的分析師，分析師普遍預期在疫情全球大流行的情況下，中國大陸企業的損失相較其他地區為少，處於相對有利位置，有望率先從疫情中復甦。
儘管達87%的大陸市場分析師預期新冠疫情將有損區內企業獲利，比例為各地區調查中最高，但大陸市場分析師普遍預計，企業受疫情衝擊的程度，將較其他地區來得溫和。
Fiona O'Neill表示，中國大陸政府在疫情爆發的較早階段實施嚴格的旅遊限制，經濟受衝擊的層面雖然較廣，但只要疫情沒有出現第二波爆發，其經濟便有望以相對較快的速度復甦，對整體企業獲利的影響也相對較少。
Fiona O'Neill也指出，隨着各國為抗疫限制旅遊及社交聚會，科技行業中的電玩遊戲及遠距辦公等領域的發展將急速竄升，科技行業的經常性收入料將增加，可緩解市場對此行業現金流的憂慮。
「科技企業也是面對疫情準備最充分的產業之一，業內大多早已建立了遠距工作文化，同時所擁有的遠距辦公設備及技術，也較其他行業完備。」Fiona O'Neill說道。
----------

▲玉山銀行攜手新加坡金融科技業者Razer Fintech，打造雙向跨境支付服務。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
瞄準東南亞電商蓬勃發展，玉山銀行宣布，攜手爭取新加坡純網銀執照的金融科技業者Razer Fintech合作，連結旗下第三方支付平台Razer Merchant Services（RMS），打造雙向跨境支付服務，讓民眾在生活場景中取得便利的金融服務。
玉山銀行表示，繼與國際支付領導業者PayPal、支付寶、微信及日本NTT Data合作後，如今看好東南亞電子商務蓬勃發展，再與科技應用領導品牌Razer Fintech合作，提供台灣消費者以多元便利的支付方式，購買全球電商的數位服務或實體商品，同時也提供台灣商戶或賣家外銷東南亞市場，透過RMS收款並將帳戶餘額提回台灣，與台灣企業共創新商機。
Razer Fintech為東南亞最具指標性的金融科技業者之一，旗下RMS前身為東南亞最大第三方支付平台MOLPay，合作電商包括全球知名品牌如Google、Facebook、Agoda、Amazon、Lazada、蝦皮、天貓、淘寶、Grab等，並已整合馬來西亞、新加坡、印尼、泰國、越南、菲律賓等多國在地化支付金流，每年處理總值數十億美元的電子支付交易。
玉山銀行信用卡暨支付金融事業處副總經理劉美玲表示，透過Razer Fintech和玉山的創新能力、用戶體驗、風險控管與豐富的跨界連結經驗，為台灣與東南亞的消費者、商戶打造安全便利的跨境支付體驗，並逐步深化各項場景金融的合作佈局。
劉美玲指出，這次透過玉山與RMS平台整合，不論國際電商進入台灣市場，或台灣電商前進東南亞，僅須透過一次性的API串接，即可連接台灣及東南亞各國在地化的支付金流服務，雙方合作項目預計在今年陸續上線。
----------------------------------------------------------------------

[44] 悠遊卡電子支付上線「嗶乘車」　董座陳亭如：持續努力讓蘋果加入

▲悠遊付記者會。（圖／記者楊亞璇攝）
記者楊亞璇／台北報導
悠遊卡公司23日宣布，電子支付「悠遊付」3月23日起全面開放註冊，未來在悠遊卡、悠遊付「一卡一付」的整合下，最為重要的功能為「嗶乘車」，只要民眾持有NFC功能之Android6.0以上手機就能感應進出雙北捷運、雙北公

▲ 新台幣兌美元今天開盤貶值1.48角。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（23）日開盤重貶1.48角、以30.45元兌1美元開出。
新台幣匯率上周五（20日）收在30.302元兌1美元，今天開盤以30.45元兌1美元開出、貶值1.48角。
截至上午9點04分，新台幣來到30.429元兌1美元，貶值1.27角。
受到市場持續恐慌影響，台股今日早盤也大幅受挫，開盤以下跌208點開出，隨後持續下挫，跌點擴大到360點以上，並跌破8900點整數關卡。
----------------------------------------------------------------------

[59] 疫情蔓延、紓困措施未通過　美股三大指數期貨皆出現跌停　

▲美股。（圖／路透社）
記者余弦妙／台北報導
美國新冠肺炎疫情蔓延，美國國會又未能就支出法案達成協議，美國三大指數期貨在台灣時間周一早上出現暴跌，甚至一度跌停。
道瓊期指截至台灣時間9點，為18109.5點，下跌930.5點或4.89%。
另外，那斯達克期指下跌310.75點或4.46%，為6658.25點。
標普500期指則跌109.38點或4.78%%，來到2179.12點。
新冠肺炎疫情持續在美國延燒，總統川普提出的經濟紓困法案卻遭民主黨參議員封殺，美股期指周一再度出現崩跌行情，跌幅並達5%，再度觸發跌停機制。
美國共和黨所提規模高達2兆美元的經濟紓困，遭民主黨參議員封殺，因民主黨認為，這項提案無法充分協助數以百萬計受疫情衝擊的美國民眾，受到提案無法過關的衝擊，美股期指今日開盤應聲重挫。
----------------------------------------------------------------------

[60] 台股失守8900點！早盤重挫361點　國安基金面臨考驗

▲ 台股。（資料照／記者湯興漢攝）
記者李瑞瑾／台北報導
新冠肺炎（COVID-19）疫情在歐美肆虐，市場人心惶惶，美股持續走低，美股期指今早一度熔斷，台股上周五雖反彈收高，但仍不敵國際股市同步下挫壓力，今（23）日開盤下跌208點、以9025點開出。
截至上午9點01分，指數來到8872點、下跌361點。
（更新）截至上午9點12分，指數一度下跌483點、最低來到8750點。


▲統一發票兌獎。（圖／記者許力方攝）
記者吳靜君／台北報導
統一發票1、2月期即將在週三（25日）開獎，民眾又可以享受兌獎的樂趣！但是別忘了喔！108年11、12月期統一發票1,000萬元特別號中獎「59647042」中獎張數共計有13張，其中有5張尚未兌領，其中花35元在7-11(統一超商）學屋門市買零食，就中千萬大獎的幸運兒，也尚未兌領。
財政部表示，去年11、12月期的中獎統一發票兌獎時間是只到5月5日，所以還未兌獎的民眾，記得要去兌領。
根據財政部所公布的資料顯示，千萬特別獎還未兌領有5張發票，台北市有1張，為台北市大安區的遠傳電信開出，消費者儲值300元點數儲值費。
新北市也有1張，則是在新北市土城區的台塑延吉加油站，加79元的汽油。桃園市則有1張，在桃園市中壢區的7-11學屋門市購買35元手摘果物就中了大獎。
台南市的1張同樣在7-11消費，在台中市南屯區黎明東門市購買照燒雞糖心蛋三明治與熱拿鐵，花費119元。
高雄市1張，則是在高雄鳥松區高雄長庚醫院的摩斯漢堡買熱抹茶拿鐵80元，就中的千萬統一發票大獎。
▲108年11、12月統一發票未兌領清冊。（表／記者吳靜君、資料來源／財政部）
另外，108年11、12月期200萬元特獎中獎號碼為「01260528」，中獎張數共計18張，有10張尚未兌領。
根據財政部的未領清冊顯示，台北市有2張，其中一張是台北市大安區的優勢領航科技開出，消費者花400元儲值「Ｇ妹遊戲」；另一張則是App Store首次開出的大獎，消費者在iTunes購買數位影音150元。基隆市也有1張，則是在基隆市七堵區的OK便利商店永富店裡，購買樂事九州岩燒海苔，花費30元。
新北市也有2張，一張則是在新北市淡水區7-11金沙門市購買菸品以及打火機，總花費為120元；另一張則是新北市中和區的在美聯社中和民利店購買台啤，花費共計154元。
宜蘭縣則也有1張，則是在宜蘭縣宜蘭市的中油廣進加油站洗車，花費70元；以及新竹縣也有1張，則是在新竹縣新埔鎮的7-11新普門市購買報紙和食品等，花費160元。台南市有1張，為在台南市新營區的全家新營車站店購買大口法式奶油起司雞排飯糰以及巧克力牛奶，花費59元。
高雄市則有2張，分別是在高雄鳥松區的成旺金香鋪購買金紙1680元；以及高雄三民區的生力美食購買麵包花了252元。財政部表示，去年的11、12月中獎的統一

▲經濟部工業局攜手工研院、榮剛、久陽等大廠，打造智能金屬產業育才平台，透過產學合作培養人才。(圖／經濟部提供)
記者林淑慧／台北報導
年後轉職潮一波接一波，為加速金屬產業邁向智慧製造，經濟部工業局攜手工研院、榮剛、久陽等大廠，打造智能金屬產業育才平台，促成企業透過產學合作模式，打造金屬產業新秀MVP，協助金屬產業拼轉型。
為達成產品高值化、製造智慧化及管理數位化，智能金屬產業育才平台協助媒合包括鋼鐵(榮剛、天聲)、扣件(久陽)、鑄造(精工、勤美)及模具(龍畿)等產業廠商，鏈結國立高雄大學、國立高雄科技大學及明志科技大學，共同培育金屬產業優質人才。
官員指出，此6家企業結合大學校院開立客製化專業課程，依企業人才需求量身訂製培育企業所需人才，同時活絡產業人才孕育橋梁，促成企業透過產學合作模式，並以企業高值化及智能化技術需求為導向，規劃專題研究。
舉例來說，榮剛材料開設「智慧聯網系統於鋼鐵廠的應用」及勤美公司「MES資訊數據分析」等智慧化相關課程；另在專題研究方面，廠商依需求提出，如天聲工業「產線數據自動收集系統」及「自動化生產履歷系統」與久陽精密「螺絲線材的智能化成形」等專題，由企業提供師資及專題指導，結合學校資源使實務與理論能夠相互結合，達到深耕育才，量穩質精之效益。
經濟部工業局表示，智能金屬產業育才平台，可望輔助企業育才時，達成縮短學用落差及解決人才缺口之效益，企業可透過政府資源強化產業與周邊大學校院的鏈結，網羅優質人才及讓學生及早認識產業，以期達成「企業預約人才，學生銜接就業，學校發展特色」的三贏局面。
----------------------------------------------------------------------

[79] 元大S＆P原油正2遭質疑　金管會：投資人自認權益受損可申訴

▲市場傳出元大證券「自肥」一事，金管會今天澄清，發布函令前並未向業者透露。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
國際油價慘跌，元大S&P原油正2（00672L）淨值低於2元，已達下市門檻，金管會19日發布函令放寬標準，暫時解除下市危機，不過市場傳出元大在發布函令當天買該檔ETF逾12萬張，引起外界質疑有「自肥」的疑慮，金管會今（21）日表示，發布前並未向業者透露，且未與元大證券有任何接觸，絕不會容許證券商藉機獲利。
市場質疑金管會的特赦令走漏風

▲綜藝天王豬哥亮入殮擁恆文創園區，園區創辦人陳振豐，再獲無罪。（圖／《ETtoday新聞雲》資料照）
記者吳銘峯／台北報導
國統集團總裁陳振豐被控涉入2000年爆發的基隆市「大水窟弊案」，一審無罪，但二審遭判刑8月。案經撤銷發回後，高等法院更一審駁回檢方上訴，改判決陳振豐無罪，依照《刑事妥速審判法》規定，檢方上訴受限，陳振豐幾乎無罪確定。對此，陳振豐發出聲明，強調本案纏訟近20年遲來的正義，終於沉冤得雪。
陳振豐身兼薇閣汽車旅館老闆、國統集團總裁、擁恆文創園區創辦人，其家族企業為股票上市公司高意建設開發公司，高意公司更是另一家上市公司基泰建設的大股東。但檢方指控，兩家公司在2000年左右開發基隆「大水窟」地區，陳振豐當時為了開發擁恆文創園區，涉嫌將9000萬元回扣，酬謝給基泰建設總經理馮先勉，馮先勉還因此遭檢方收押禁見，陳振豐最後被依照「背信罪」提起公訴。
陳振豐一審獲判無罪，但二審改認定有罪，判刑8月。上訴最高法院後，最高法院撤銷發回，全案由高等法院更一審，更一審最後以查無實證為由，駁回檢方上訴，維持陳振豐的一審無罪判決。依照依照《刑事妥速審判法》規定，檢方上訴第三審受限，陳振豐幾乎無罪確定。
對此，國統集團發布聲明，強調本案纏訟近20年，終於確保陳振豐清白，遲來的正義再次證明，國統公司開發擁恆文創園區一切程序合法、合理、合情。集團發言人王志欽表示，沉冤昭雪，肯定了陳振豐及國統公司致力於擁恆文創園區的經營管理，恪遵法規，嚴守分際。未來集團將更以「家」為核心理念，打造「來者以遊，生者以慰，逝者以安」的人間淨土。
----------------------------------------------------------------------

[90] 快訊／美股開盤道瓊「綠轉紅」　再度跌破2萬點

▲美股再度跌破2萬點。（圖／達志影像／美聯社）
記者鄒鎮宇／綜合報導
美股三大指周五開盤，道瓊工業指數開盤一度上漲逾170.23點或0.85％，不久下跌114.76點，約0.57%，跌至19933.10點；標普500指和那斯達克綜合指數漲幅一度都超過2％，但標普轉綠下跌，報2381.75點。
美股開盤由科技股帶動，蘋果漲1.4%、微軟漲2.26%、亞馬遜漲3.1%、谷歌漲0.8%、臉書漲3.5%，道指一度漲超300點，但隨即紅轉綠下跌。
►酒精乾洗手下殺3折！